<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/5%F0%9F%8F%86%F0%9F%9B%92Shop_Final_v4_Team_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🛒Shop</font>](https://www.kaggle.com/competitions/26jun23jh-shop/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [1]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [2]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23jh-shop    # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23jh-shop
100% 9.98M/9.98M [00:00<00:00, 242MB/s]
Using competition: 26jun23jh-shop
  teamId  teamName             submissionDate       score    
--------  -------------------  -------------------  -------  
10631355  Group 9              2023-07-08 16:44:24  0.99324  
10638614  Alayna Stepp         2023-07-06 01:34:40  0.99272  
10638830  Team 13              2023-07-08 23:34:21  0.99008  
10652071  Ryan Dombrowski      2023-07-09 12:06:17  0.98744  
10644064  Group 2              2023-07-09 09:33:04  0.98732  
10654893  NeilJoshi            2023-07-09 05:36:25  0.98684  
10639155  Jooyoung Park        2023-07-09 06:36:04  0.98672  
10633911  Group 11             2023-07-09 03:37:52  0.98640  
10638103  Group 15             2023-07-08 03:16:47  0.98588  
10635224  Margarita Prikhodko  2023-07-09 04:21:31  0.98256  
10642846  Group 14             2023-07-08 21:37:22  0.98232  
10637839  Larry Wal

In [3]:
!pip -q install -U tensorflow_addons > log

In [4]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 6.5 s, sys: 608 ms, total: 7.11 s
Wall time: 11.2 s


In [5]:
df = pd.read_csv('XY_Shop.csv'); df

,Adm,AdmDur,Inf,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd,Rev
0,0,0.00,0,0.0,18,132.99,3.82e-02,5.45e-02,0.0,0.0,4,3,1,1,2,0,1,NaN
1,1,0.00,0,0.0,37,1150.20,1.25e-03,3.03e-02,0.0,0.0,11,2,2,4,2,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499998,0,0.00,0,0.0,27,1185.14,0.00e+00,1.59e-03,0.0,0.0,5,2,2,2,3,0,1,0.0
499999,6,51.36,0,0.0,59,1898.21,0.00e+00,3.22e-03,0.0,0.0,12,2,2,2,1,0,0,0.0


In [6]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Adm      500000 non-null  int64  
 1   AdmDur   500000 non-null  float64
 2   Inf      500000 non-null  int64  
 3   InfDur   500000 non-null  float64
 4   Prd      500000 non-null  int64  
 5   PrdDur   500000 non-null  float64
 6   BncRt    500000 non-null  float64
 7   ExtRt    500000 non-null  float64
 8   PgVal    500000 non-null  float64
 9   SpclDay  500000 non-null  float64
 10  Mo       500000 non-null  int64  
 11  OS       500000 non-null  int64  
 12  Bsr      500000 non-null  int64  
 13  Rgn      500000 non-null  int64  
 14  TfcTp    500000 non-null  int64  
 15  VstTp    500000 non-null  int64  
 16  Wkd      500000 non-null  int64  
 17  Rev      450000 non-null  float64
dtypes: float64(8), int64(10)
memory usage: 68.7 MB


In [7]:
vX = df.query('Rev!=Rev').drop('Rev', axis=1)  # slice a test sample
tXY = df.query('Rev==Rev')                     # slice training sample
tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
NumFeatures = list(tX.select_dtypes(include='float').columns)
print('Numeric features: ', NumFeatures)       # numeric/quantitative feature names

Numeric features:  ['AdmDur', 'InfDur', 'PrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay']


In [8]:
# def ScatterCorrHist(df):
#   def corrdot(*args, **kwargs):
#     # credit: https://stackoverflow.com/questions/48139899
#     corr_r = args[0].corr(args[1], 'pearson')
#     corr_text = f"{corr_r:2.2f}".replace("0.", ".")
#     ax = plt.gca();
#     ax.set_axis_off();
#     msz = abs(corr_r) * 5000   # marker size
#     fsz = abs(corr_r) * 40 + 5 # font size
#     ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
#     ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

#   sns.set(style='white', font_scale=.8);
#   g = sns.PairGrid(df, aspect=1, diag_sharey=False);
#   g.fig.set_size_inches(20,10)
#   g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
#   g.map_diag(sns.histplot, kde_kws={'color':'black'});
#   g.map_upper(corrdot);
#   g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
#   _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
#   _ = plt.show();

# df_ = tXY.loc[(tXY[NumFeatures]>0).all(axis=1), NumFeatures+['Rev']].sample(n=100, random_state=0)
# # df_ = df.select_dtypes(include='float').query('AdmDur>0 and InfDur>0 and PrdDur>0 and BncRt>0 and ExtRt>0 and PgVal>0 and SpclDay>0').sample(n=100, random_state=0)
# ScatterCorrHist(df_)

In [9]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

Things that helped improve accuracy
* There are 18 colums in the dataset. 10 of them are integer and 8 of them are float. Only floats were considered and int columns are ignored. I have expanded feature set to from 10 to 18 features by includng int columns as well.
* I checked the missing values for imputation. There are no missing values
* Feature Selection: Sometimes, not all features are equally informative. I have tried techniques like Recursive Feature Elimination, SelectKBest, or feature importance from tree-based models to select the most relevant features. I used selectKBest
Normalize the numeric features to 0 to 1 scale


Things that did not help improve accuracy
*. Balancing the dataset. The dataset is highly imbalanced. Imbalance ration i s 0.21. Hence I attempted to balance the dataset using SMOTE. However that has reduced accuracy fby 50%


In [10]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [11]:
tX, tY = tXY.drop('Rev', axis=1), tXY.Rev      # split into training I/O
# Convert integer columns to float to expand feature set
int_columns = tX.select_dtypes(include=['int']).columns
tX[int_columns] = tX[int_columns].astype(float)
tX.info
vX[int_columns] = vX[int_columns].astype(float)
tX.info

NumFeatures = list(tX.select_dtypes(include='float').columns)
print('Numeric features: ', NumFeatures)       # numeric/quantitative feature names


<bound method DataFrame.info of         Adm  AdmDur  Inf  InfDur   Prd   PrdDur  BncRt     ExtRt  PgVal  \
50000   0.0    0.00  0.0     0.0   1.0     0.00   0.16  1.15e-01    0.0   
50001   7.0  116.19  0.0     0.0  79.0  2683.58   0.00  1.91e-03    0.0   
...     ...     ...  ...     ...   ...      ...    ...       ...    ...   
499998  0.0    0.00  0.0     0.0  27.0  1185.14   0.00  1.59e-03    0.0   
499999  6.0   51.36  0.0     0.0  59.0  1898.21   0.00  3.22e-03    0.0   

        SpclDay    Mo   OS  Bsr  Rgn  TfcTp  VstTp  Wkd  
50000       0.0  12.0  2.0  2.0  1.0    9.0    0.0  0.0  
50001       0.0   4.0  2.0  3.0  2.0    2.0    1.0  0.0  
...         ...   ...  ...  ...  ...    ...    ...  ...  
499998      0.0   5.0  2.0  2.0  2.0    3.0    0.0  1.0  
499999      0.0  12.0  2.0  2.0  2.0    1.0    0.0  0.0  

[450000 rows x 17 columns]>

<bound method DataFrame.info of         Adm  AdmDur  Inf  InfDur   Prd   PrdDur  BncRt     ExtRt  PgVal  \
50000   0.0    0.00  0.0     0.0   1.0     0.00   0.16  1.15e-01    0.0   
50001   7.0  116.19  0.0     0.0  79.0  2683.58   0.00  1.91e-03    0.0   
...     ...     ...  ...     ...   ...      ...    ...       ...    ...   
499998  0.0    0.00  0.0     0.0  27.0  1185.14   0.00  1.59e-03    0.0   
499999  6.0   51.36  0.0     0.0  59.0  1898.21   0.00  3.22e-03    0.0   

        SpclDay    Mo   OS  Bsr  Rgn  TfcTp  VstTp  Wkd  
50000       0.0  12.0  2.0  2.0  1.0    9.0    0.0  0.0  
50001       0.0   4.0  2.0  3.0  2.0    2.0    1.0  0.0  
...         ...   ...  ...  ...  ...    ...    ...  ...  
499998      0.0   5.0  2.0  2.0  2.0    3.0    0.0  1.0  
499999      0.0  12.0  2.0  2.0  2.0    1.0    0.0  0.0  

[450000 rows x 17 columns]>

Numeric features:  ['Adm', 'AdmDur', 'Inf', 'InfDur', 'Prd', 'PrdDur', 'BncRt', 'ExtRt', 'PgVal', 'SpclDay', 'Mo', 'OS', 'Bsr', 'Rgn', 'TfcTp', 'VstTp', 'Wkd']


In [12]:
tX_in = tXY[NumFeatures][:30000]
tY_in = tXY.Rev[:30000]
vX_in = vX

In [13]:
tX_in.isna().sum()

Adm       0
AdmDur    0
         ..
VstTp     0
Wkd       0
Length: 17, dtype: int64

In [14]:
# Data Preprocessing
scaler = StandardScaler()
tX_in_scaled = scaler.fit_transform(tX_in)
vX_in_scaled = scaler.fit_transform(vX_in)


In [15]:
tX_in_scaled.size

510000

In [16]:
# Feature Selection
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
# Fit the selector on the training data
selector = SelectKBest(score_func=f_classif, k=10)
selector.fit(tX_in_scaled, tY_in)

# Transform both training and validation/test data
tX_selected = selector.transform(tX_in_scaled)
vX_selected = selector.transform(vX_in_scaled)  # Assuming vX_scaled is your scaled validation/test data


SelectKBest()

In [17]:
vX_selected.shape

(50000, 10)

In [18]:
#Check data impbalance
# Count the number of instances in each class
class_counts = np.bincount(tY.astype(int))

# Calculate the ratio of the minority class to the majority class
imbalance_ratio = min(class_counts) / max(class_counts)

# Print the results
print(f"Number of instances in each class: {class_counts}")
print(f"Imbalance ratio: {imbalance_ratio}")

Number of instances in each class: [349140 100860]
Imbalance ratio: 0.2888812510740677


In [19]:
#Balance the dataset. This reduced accuracy by 50%. Henced commeneted the code
#from imblearn.over_sampling import SMOTE
#smote = SMOTE()
#tX_balanced, tY_balanced = smote.fit_resample(tX_selected, tY_in)



## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

 I tried changing following hyperparameters.

 The hyperparameters tuning that helped raising accuracy above baseline
* Model Architecture: I experimented with different neural network architectures. This includes changing the number of layers, the number of neurons in each layer, and the activation functions. I tried adding new layers and it has helped improving accuracy
* Batch Size and Epochs: I experimented with different batch sizes and number of epochs. Sometimes training with a smaller batch size can yield a better generalization.
* Regularization: I have added regularization to the layers.
* Polynomial features helped improve the performance of a model by capturing interactions between features and considering them in higher-dimensional space.
* I have added dropout layer to prevent overfitting

I have tried few new models like XGB, Random Forest, Decision Trees and CNN

| Model   |          Accuracy |
|---------|----------|
| Sequential ANN   | 82%        |
| CNN     | 81%     |
| XGB | 86%     |
| Random Forest  | 89%     |
| Decision Tree  | 84%     |


  
  The hyper parameters tuning that did not help improve baseline accuracy
 * Changing learning rate
 * Changing size of neurons in hidden layers

 I have also tried changing model arhitecture to CNN , but that did not help improve accuracy


In [20]:
tX_selected.shape[1]

10

In [21]:
# Polynomial Features
poly = PolynomialFeatures(degree=2)  # You can change the degree
tX_poly = poly.fit_transform(tX_selected)
vX_poly = poly.fit_transform(vX_selected)



#Below is a Sequenctial ANN model with regualrization, drop out and more layers . This gives accuracy of 82%

In [22]:

%%time
from tensorflow.keras.layers import Dense, Flatten, Dropout
tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.RandomNormal(seed=0)



m = keras.models.Sequential([
    Flatten(input_shape=[tX_poly.shape[1]], name='input'),
    Dense(16, activation="relu", kernel_initializer=Init, kernel_regularizer=regularizers.l2(0.0001), name='hidden4'),# I added this layer
    Dropout(0.005),
    Dense(50, activation="relu", kernel_initializer=Init, kernel_regularizer=regularizers.l2(0.0001),  name='hidden'),
    Dropout(0.005),
    Dense(32, activation="relu", kernel_initializer=Init, kernel_regularizer=regularizers.l2(0.0001), name='hidden2'),   # I added this layer
    Dropout(0.005),
    Dense(16, activation="relu", kernel_initializer=Init, kernel_regularizer=regularizers.l2(0.0001), name='hidden3'),   # I added this layer
    Dropout(0.005),
    Dense(1, activation='sigmoid', kernel_initializer=Init, name='output')])
m.summary()
m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"])
hist = m.fit(tX_poly, tY_in, epochs=15, validation_split=0.3, batch_size=32)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 66)                0         
                                                                 
 hidden4 (Dense)             (None, 16)                1072      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 hidden (Dense)              (None, 50)                850       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 hidden2 (Dense)             (None, 32)                1632      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0

#Let try with XGB Model.
#Observation: XGB Model performs a lot better as compared t sequential model
# Accuracy is up to 86%

In [23]:
'''
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures

# Polynomial Features
#poly = PolynomialFeatures(degree=2)  # You can change the degree
#tX_poly = poly.fit_transform(tX)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)

# Create the XGBoost model
m = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create the XGBoost model with custom hyperparameters

m = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,
    max_depth=3,
    learning_rate=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1
)

# Fit the model on the training data
m.fit(X_train, y_train)

# Predict on the test set
y_pred = m.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
'''


"\nimport xgboost as xgb\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.preprocessing import PolynomialFeatures\n\n# Polynomial Features\n#poly = PolynomialFeatures(degree=2)  # You can change the degree\n#tX_poly = poly.fit_transform(tX)\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)\n\n# Create the XGBoost model\nm = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')\n\n# Create the XGBoost model with custom hyperparameters\n\nm = xgb.XGBClassifier(\n    use_label_encoder=False,\n    eval_metric='logloss',\n    n_estimators=100,\n    max_depth=3,\n    learning_rate=0.01,\n    subsample=0.8,\n    colsample_bytree=0.8,\n    gamma=0,\n    reg_alpha=0,\n    reg_lambda=1,\n    scale_pos_weight=1\n)\n\n# Fit the model on the training data\nm.fit(X_train, y_train)\n\n# Predict on the test set\ny_pred = m.predict

#Lets try RandomForest Model

In [24]:
'''
%%time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split



# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)

# Create the Random Forest model with custom hyperparameters


# Create the model
#m = RandomForestClassifier(n_estimators=100, random_state=0)

m = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='auto',
    bootstrap=True,
    random_state=0
)

m = RandomForestClassifier(max_features='auto', n_estimators=200, random_state=0)



# Fit the model on the training data
m.fit(X_train, y_train)

# Predict on the test set
y_pred = m.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
'''

"\n%%time\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.model_selection import train_test_split\n\n\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)\n\n# Create the Random Forest model with custom hyperparameters\n\n\n# Create the model\n#m = RandomForestClassifier(n_estimators=100, random_state=0)\n\nm = RandomForestClassifier(\n    n_estimators=100,\n    max_depth=None,\n    min_samples_split=2,\n    min_samples_leaf=1,\n    max_features='auto',\n    bootstrap=True,\n    random_state=0\n)\n\nm = RandomForestClassifier(max_features='auto', n_estimators=200, random_state=0)\n\n\n\n# Fit the model on the training data\nm.fit(X_train, y_train)\n\n# Predict on the test set\ny_pred = m.predict(X_test)\n\n# Calculate and print the accuracy\naccuracy = accuracy_score(y_test, y_pred)\nprint(f'Test Accuracy: {accuracy}')\n"

#Random Forest model. The score is same as above (88.33%)

In [25]:
#Random Forest with CV
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import PolynomialFeatures
import numpy as np


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)

# Create the Random Forest model with custom hyperparameters
m = RandomForestClassifier(max_features='auto', n_estimators=200, random_state=0)

# Perform cross-validation and compute the mean score
cv_scores = cross_val_score(m, X_train, y_train, cv=5)
mean_cv_score = np.mean(cv_scores)

# Print the mean cross-validation score
print(f'Mean Cross-Validation Score: {mean_cv_score}')

# Fit the model on the training data
m.fit(X_train, y_train)

# Predict on the test set
y_pred = m.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
'''

"\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import cross_val_score, train_test_split\nfrom sklearn.preprocessing import PolynomialFeatures\nimport numpy as np\n\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)\n\n# Create the Random Forest model with custom hyperparameters\nm = RandomForestClassifier(max_features='auto', n_estimators=200, random_state=0)\n\n# Perform cross-validation and compute the mean score\ncv_scores = cross_val_score(m, X_train, y_train, cv=5)\nmean_cv_score = np.mean(cv_scores)\n\n# Print the mean cross-validation score\nprint(f'Mean Cross-Validation Score: {mean_cv_score}')\n\n# Fit the model on the training data\nm.fit(X_train, y_train)\n\n# Predict on the test set\ny_pred = m.predict(X_test)\n\n# Calculate and print the accuracy\naccuracy = accuracy_score(y_test, y_pred)\nprint(f'Test Accuracy: {accuracy}')\n"

#Let's try decision Tree model/
## Accuracy is 84%

In [26]:
'''
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PolynomialFeatures



# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)

# Create the Decision Tree model
m = DecisionTreeClassifier(random_state=0)

# Fit the model on the training data
m.fit(X_train, y_train)

# Predict on the test set
y_pred = m.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')
'''

"\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.preprocessing import PolynomialFeatures\n\n\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)\n\n# Create the Decision Tree model\nm = DecisionTreeClassifier(random_state=0)\n\n# Fit the model on the training data\nm.fit(X_train, y_train)\n\n# Predict on the test set\ny_pred = m.predict(X_test)\n\n# Calculate and print the accuracy\naccuracy = accuracy_score(y_test, y_pred)\nprint(f'Test Accuracy: {accuracy}')\n"

In [27]:
'''
#GridSearchCV to find best params of RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid, train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
import numpy as np


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

# Create a Random Forest model
rf_model = RandomForestClassifier(random_state=0)

# Create a list to store the average cross-validation score for each combination of parameters
scores = []

# Loop over each combination of parameters
for params in ParameterGrid(param_grid):

    # Set the parameters on the model
    rf_model.set_params(**params)

    # Perform cross-validation and compute the mean score
    cv_scores = cross_val_score(rf_model, X_train, y_train, cv=3)
    mean_cv_score = np.mean(cv_scores)

    # Print the parameters and the mean score
    print(f'Parameters: {params}')
    print(f'Mean CV Score: {mean_cv_score}')

    # Append the score to the scores list
    scores.append(mean_cv_score)

# Get the parameters that gave the highest mean cross-validation score
best_params = ParameterGrid(param_grid)[np.argmax(scores)]
print(f'Best parameters: {best_params}')

# Fit the model on the training data using the best parameters
best_rf_model = RandomForestClassifier(**best_params, random_state=0)
best_rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_rf_model.predict(X_test)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy}')

'''

"\n#GridSearchCV to find best params of RandomForest\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import ParameterGrid, train_test_split, cross_val_score\nfrom sklearn.preprocessing import PolynomialFeatures\nimport numpy as np\n\n\n# Split the data into training and test sets\nX_train, X_test, y_train, y_test = train_test_split(tX_poly, tY_in, test_size=0.2, random_state=42)\n\n# Define the grid of hyperparameters to search\nparam_grid = {\n    'n_estimators': [100, 200],\n    'max_depth': [None, 10],\n    'min_samples_split': [2, 5],\n    'min_samples_leaf': [1, 2],\n    'max_features': ['auto', 'sqrt']\n}\n\n# Create a Random Forest model\nrf_model = RandomForestClassifier(random_state=0)\n\n# Create a list to store the average cross-validation score for each combination of parameters\nscores = []\n\n# Loop over each combination of parameters\nfor params in ParameterGrid(param_grid):\n\n    # Set the parameters on the model\n    rf_model.set_par

# Changed Model Architecture to CNN . However accuracy did not improve.

In [28]:

'''
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler

# Assuming tX_scaled and tY are your preprocessed features and labels

# Reshaping the data to be 3D as needed for CNN
tX_reshaped = tX_poly.reshape(tX_poly.shape[0], tX_poly.shape[1], 1)

# Model
model = Sequential()

# Adding Convolutional Layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(tX_reshaped.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

# Adding Dense Layers
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(tX_reshaped, tY, epochs=10, batch_size=32, validation_split=0.2)
'''

"\nimport tensorflow as tf\nfrom tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D\nfrom tensorflow.keras import Sequential\nfrom sklearn.preprocessing import StandardScaler\n\n# Assuming tX_scaled and tY are your preprocessed features and labels\n\n# Reshaping the data to be 3D as needed for CNN\ntX_reshaped = tX_poly.reshape(tX_poly.shape[0], tX_poly.shape[1], 1)\n\n# Model\nmodel = Sequential()\n\n# Adding Convolutional Layers\nmodel.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(tX_reshaped.shape[1], 1)))\nmodel.add(MaxPooling1D(pool_size=2))\nmodel.add(Flatten())\n\n# Adding Dense Layers\nmodel.add(Dense(50, activation='relu'))\nmodel.add(Dense(1, activation='sigmoid'))\n\n# Compile the model\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n\n# Fit the model\nmodel.fit(tX_reshaped, tY, epochs=10, batch_size=32, validation_split=0.2)\n"

In [29]:
vX.reset_index()

,index,Adm,AdmDur,Inf,InfDur,Prd,PrdDur,BncRt,ExtRt,PgVal,SpclDay,Mo,OS,Bsr,Rgn,TfcTp,VstTp,Wkd
0,0,0.0,0.00,0.0,0.0,18.0,132.99,3.82e-02,5.45e-02,0.00,0.0,4.0,3.0,1.0,1.0,2.0,0.0,1.0
1,1,1.0,0.00,0.0,0.0,37.0,1150.20,1.25e-03,3.03e-02,0.00,0.0,11.0,2.0,2.0,4.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49998,49998,4.0,357.05,0.0,0.0,27.0,1278.00,0.00e+00,6.81e-03,14.54,0.0,11.0,2.0,2.0,5.0,2.0,0.0,0.0
49999,49999,0.0,0.00,0.0,0.0,26.0,1646.42,7.55e-03,3.25e-02,0.00,0.0,2.0,2.0,2.0,4.0,3.0,0.0,1.0


In [30]:
vX.shape

(50000, 17)

In [31]:
vX.index+1

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            49991, 49992, 49993, 49994, 49995, 49996, 49997, 49998, 49999,
            50000],
           dtype='int64', length=50000)

In [32]:
pY = pd.DataFrame(m.predict(vX_poly),index=vX.index+1, columns=['Rev'])
ToCSV(pY.round(0).astype(int), 'Ravindra_shop_v5')
tXY.Rev.value_counts()/len(tXY)                # distribution of training target level
pY.round(0).astype(int).value_counts()/len(pY) # distribution of test target level

1563/1563 [==============================] - 3s 2ms/step


0.0    0.78
1.0    0.22
Name: Rev, dtype: float64

Rev
0      0.86
1      0.14
dtype: float64

# **References:**

Textbook:

Hastie, T., Tibshirani, R., & Friedman, J. (2009). The elements of statistical learning: data mining, inference, and prediction. Springer Science & Business Media.


Online Documentation:
TensorFlow. (n.d.). TensorFlow Core.  from https://www.tensorflow.org/api_docs


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [33]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 93 sec > 60 sec limit!!!


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Incorporate categorical features (appropriately encoded)
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?

**Training observations**
1. Try clustering methods to remove similar observations. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain: how should output relate to features? How do month or weekend impact users' buying activity?
  1. User Agent [&#127910;](https://www.youtube.com/results?search_query=user+agent+browser), Google Analytics [&#127910;](https://www.youtube.com/results?search_query=google+analytics), tracking online shopping intent [&#127910;](https://www.youtube.com/results?search_query=tacking+online+shopping+intent), [📄](https://scholar.google.com/scholar?q=tracking+online+shopping+intent)